## Project Overview

### Problem statement

SyriaTel, a telecommunications company, is facing a high churn rate, with many customers discontinuing their services and switching to competitors. By analyzing the dataset, SyriaTel aims to gain insights into factors associated with high customer churn, leading to significant revenue loss

Objectives are:

1. Identify the factors that are most likely to lead to customer churn.

2. Develop a model that can accurately predict which customers are at risk of churning.

3. Take proactive steps to retain customers who are at risk of churning.

My stakeholders are:

1. SyriaTel company

2. Potential Investors

3. Partners of SyriaTel 

By analyzing customer data and identifying key factors contributing to churn, our project offers valuable insights for telecom businesse. These insights can be used to implement targeted strategies to improve customer satisfaction, reduce churn rates, and ultimately increase revenue. 



## Data understanding

We source our data from Kaggle: Churn in Telecom's dataset. These data sources are well-suited for this project because they provide a comprehensive understanding of customer behavior, service usage, and factors that may contribute to churn. 